## Imbunatatire: Utilizare Focal Loss

Pentru a crește performanța **(în special acel F1-Score care era mic la clasele rare)**, am ales sa folosesc o functie diferita de loss - *Focal Loss*.
Aceasta este cea mai eficientă metodă algoritmică pentru a rezolva problema dezechilibrului de clasă în medicină (unde cazurile de "General Medicine" sunt multe, iar cele de "Oncology" sunt rare).

Focal Loss (introdusă inițial pentru detecția obiectelor) modifică funcția de pierdere astfel încât să reducă drastic importanța exemplelor pe care modelul le clasifică deja corect cu încredere mare și să se concentreze pe exemplele **"dificile"** (cele pe care le greșește).

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2.0, reduction='mean'):
        """
        alpha: Greutățile claselor (Tensor-ul calculat anterior cu compute_class_weight)
        gamma: Factorul de focalizare (standard este 2.0). Cu cât e mai mare, cu atât penalizează mai mult exemplele ușoare.
        """
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        # Calculează Cross Entropy Loss standard
        ce_loss = F.cross_entropy(inputs, targets, reduction='none', weight=self.alpha)

        # Calculează probabilitatea asociată clasei corecte (pt)
        pt = torch.exp(-ce_loss)

        # Calculează Focal Loss: (1 - pt)^gamma * CE
        focal_loss = ((1 - pt) ** self.gamma) * ce_loss

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss